<a href="https://colab.research.google.com/github/ShreyasHavaldar7/DL-Applications/blob/master/Basic_GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [3]:
pip install torch-geometric

     |████████████████████████████████| 122kB 2.9MB/s 
     |████████████████████████████████| 348kB 39.8MB/s 
     |████████████████████████████████| 51kB 23.3MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.3.1-cp36-none-any.whl size=200758 sha256=fd579233964590f386c50f2e9a36f59036c3b2674d3bb2da20bcf4c662cd0f42
  Stored in directory: /root/.cache/pip/wheels/89/e3/13/f4a170e5aaf638316f4b8445cc3a6aaae7bc7a0ad3b504c862
  Created wheel for plyfile: filename=plyfile-0.7-cp36-none-any.whl size=8239 sha256=a3a04ec791acae37d218d188cdec75db5830a91e523dd555594cbe42c1c06138
  Stored in directory: /root/.cache/pip/wheels/91/3e/ee/e5630ef0fd53cedaa6e911ba27e8b40fff034388d1f264bb92
Successfully built torch-geometric plyfile


In [4]:
!pip install torch-scatter
!pip install torch-sparse
!pip install torch-cluster
!pip install torch-spline-conv 

  Created wheel for torch-scatter: filename=torch_scatter-1.3.1-cp36-cp36m-linux_x86_64.whl size=2722840 sha256=149fa008fba3a4f6965b68727a61f96f82e1fb822fd9e46a99d9b31a9e4d1130
  Stored in directory: /root/.cache/pip/wheels/7f/21/0b/c42fa9353ceec5e87464599e470a03e4250ec667b4a392fa7d
Successfully built torch-scatter
  Created wheel for torch-sparse: filename=torch_sparse-0.4.0-cp36-cp36m-linux_x86_64.whl size=3553462 sha256=847e26b4d7af4f7d06a6abbb09efc0a49bb5b4791e033c5f1d5cdc8caa125838
  Stored in directory: /root/.cache/pip/wheels/9d/83/0a/38ea460df5586a075b877fe089619e5238487712a0645940bd
Successfully built torch-sparse
  Created wheel for torch-cluster: filename=torch_cluster-1.4.4-cp36-cp36m-linux_x86_64.whl size=14445584 sha256=2576b0c3a39723f65953e44edde95c20f50f7d88c7b526012cb52bd1a5c8c3ce
  Stored in directory: /root/.cache/pip/wheels/20/7b/ab/b3e266920055d1e51988f93a99ef8df62e399b234c8d50527f
Successfully built torch-cluster
  Created wheel for torch-spline-conv: filename=tor

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [0]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = GCNConv(dataset.num_node_features, 16)
    self.conv2 = GCNConv(16, dataset.num_classes)
  def forward(self, data):
    x, edge_index = data.x, data.edge_index

    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = F.dropout(x, training = self.training)
    x = self.conv2(x, edge_index)

    return F.log_softmax(x, dim=1)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [10]:
!pip install gputil
!pip install psutil
!pip install humanize

import GPUtil as GPU
import os
import humanize
import psutil


GPUs = GPU.getGPUs()
gpu= GPUs[0]

def printer():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available),
          " | Proc size: " + humanize.naturalsize(process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(
        gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
    
   
printer()  
#checking alloted gpu by colab

Gen RAM Free: 12.7 GB  | Proc size: 331.0 MB
GPU RAM Free: 11430MB | Used: 11MB | Util   0% | Total 11441MB


In [11]:
!nvidia-smi

Mon Sep 30 16:02:15 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |     11MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [21]:
import matplotlib.pyplot as plt
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay= 7e-4)

model.train()
for epoch in range(500):
  optimizer.zero_grad()
  out = model(data)
  loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
  print('Epoch: {} | Loss: {:.7f} ' .format(epoch, loss.item()) )
  loss.backward()
  optimizer.step()


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

Epoch: 0 | Loss: 1.9577887 
Epoch: 1 | Loss: 1.8575459 
Epoch: 2 | Loss: 1.7403291 
Epoch: 3 | Loss: 1.6110764 
Epoch: 4 | Loss: 1.4570453 
Epoch: 5 | Loss: 1.3526889 
Epoch: 6 | Loss: 1.2661147 
Epoch: 7 | Loss: 1.1085765 
Epoch: 8 | Loss: 1.0026431 
Epoch: 9 | Loss: 0.8856771 
Epoch: 10 | Loss: 0.7597647 
Epoch: 11 | Loss: 0.7121249 
Epoch: 12 | Loss: 0.6096970 
Epoch: 13 | Loss: 0.5733217 
Epoch: 14 | Loss: 0.4558171 
Epoch: 15 | Loss: 0.4122219 
Epoch: 16 | Loss: 0.3761725 
Epoch: 17 | Loss: 0.3573345 
Epoch: 18 | Loss: 0.2760734 
Epoch: 19 | Loss: 0.2775587 
Epoch: 20 | Loss: 0.2796456 
Epoch: 21 | Loss: 0.1974594 
Epoch: 22 | Loss: 0.2105751 
Epoch: 23 | Loss: 0.2042525 
Epoch: 24 | Loss: 0.1490907 
Epoch: 25 | Loss: 0.1377060 
Epoch: 26 | Loss: 0.1320486 
Epoch: 27 | Loss: 0.1356113 
Epoch: 28 | Loss: 0.1221028 
Epoch: 29 | Loss: 0.1282734 
Epoch: 30 | Loss: 0.0841434 
Epoch: 31 | Loss: 0.0878437 
Epoch: 32 | Loss: 0.0949070 
Epoch: 33 | Loss: 0.0763887 


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

Epoch: 34 | Loss: 0.0809992 
Epoch: 35 | Loss: 0.0726606 
Epoch: 36 | Loss: 0.0850056 
Epoch: 37 | Loss: 0.0614818 
Epoch: 38 | Loss: 0.0705231 
Epoch: 39 | Loss: 0.0584075 
Epoch: 40 | Loss: 0.0603376 
Epoch: 41 | Loss: 0.0710098 
Epoch: 42 | Loss: 0.0544165 
Epoch: 43 | Loss: 0.0764778 
Epoch: 44 | Loss: 0.0724207 
Epoch: 45 | Loss: 0.0430465 
Epoch: 46 | Loss: 0.0598077 
Epoch: 47 | Loss: 0.0595491 
Epoch: 48 | Loss: 0.0718925 
Epoch: 49 | Loss: 0.0624415 
Epoch: 50 | Loss: 0.0503396 
Epoch: 51 | Loss: 0.0389335 
Epoch: 52 | Loss: 0.0494265 
Epoch: 53 | Loss: 0.0557842 
Epoch: 54 | Loss: 0.0470850 
Epoch: 55 | Loss: 0.0732904 
Epoch: 56 | Loss: 0.0465756 
Epoch: 57 | Loss: 0.0440508 
Epoch: 58 | Loss: 0.0476917 
Epoch: 59 | Loss: 0.0518243 
Epoch: 60 | Loss: 0.0536235 
Epoch: 61 | Loss: 0.0739566 
Epoch: 62 | Loss: 0.0495970 
Epoch: 63 | Loss: 0.0550396 
Epoch: 64 | Loss: 0.0535775 
Epoch: 65 | Loss: 0.0338862 
Epoch: 66 | Loss: 0.0498728 
Epoch: 67 | Loss: 0.0496974 


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

...

/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

Epoch: 416 | Loss: 0.0330308 
Epoch: 417 | Loss: 0.0283984 
Epoch: 418 | Loss: 0.0312261 
Epoch: 419 | Loss: 0.0234658 
Epoch: 420 | Loss: 0.0344283 
Epoch: 421 | Loss: 0.0182639 
Epoch: 422 | Loss: 0.0200396 
Epoch: 423 | Loss: 0.0165786 
Epoch: 424 | Loss: 0.0266909 
Epoch: 425 | Loss: 0.0242126 
Epoch: 426 | Loss: 0.0355040 
Epoch: 427 | Loss: 0.0217279 
Epoch: 428 | Loss: 0.0447973 
Epoch: 429 | Loss: 0.0268018 
Epoch: 430 | Loss: 0.0290860 
Epoch: 431 | Loss: 0.0197306 
Epoch: 432 | Loss: 0.0312125 
Epoch: 433 | Loss: 0.0306466 
Epoch: 434 | Loss: 0.0237584 
Epoch: 435 | Loss: 0.0235408 
Epoch: 436 | Loss: 0.0236894 
Epoch: 437 | Loss: 0.0245032 
Epoch: 438 | Loss: 0.0182835 
Epoch: 439 | Loss: 0.0162092 
Epoch: 440 | Loss: 0.0140009 
Epoch: 441 | Loss: 0.0331678 
Epoch: 442 | Loss: 0.0215831 
Epoch: 443 | Loss: 0.0157165 
Epoch: 444 | Loss: 0.0231237 
Epoch: 445 | Loss: 0.0256047 
Epoch: 446 | Loss: 0.0296210 
Epoch: 447 | Loss: 0.0368349 
Epoch: 448 | Loss: 0.0201951 


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

Epoch: 449 | Loss: 0.0175816 
Epoch: 450 | Loss: 0.0233595 
Epoch: 451 | Loss: 0.0316592 
Epoch: 452 | Loss: 0.0243159 
Epoch: 453 | Loss: 0.0233266 
Epoch: 454 | Loss: 0.0261276 
Epoch: 455 | Loss: 0.0146360 
Epoch: 456 | Loss: 0.0211296 
Epoch: 457 | Loss: 0.0229888 
Epoch: 458 | Loss: 0.0266111 
Epoch: 459 | Loss: 0.0393082 
Epoch: 460 | Loss: 0.0287311 
Epoch: 461 | Loss: 0.0321859 
Epoch: 462 | Loss: 0.0394234 
Epoch: 463 | Loss: 0.0184755 
Epoch: 464 | Loss: 0.0249810 
Epoch: 465 | Loss: 0.0133514 
Epoch: 466 | Loss: 0.0219869 
Epoch: 467 | Loss: 0.0241445 
Epoch: 468 | Loss: 0.0210009 
Epoch: 469 | Loss: 0.0242626 
Epoch: 470 | Loss: 0.0155393 
Epoch: 471 | Loss: 0.0272437 
Epoch: 472 | Loss: 0.0282828 
Epoch: 473 | Loss: 0.0191606 
Epoch: 474 | Loss: 0.0264697 
Epoch: 475 | Loss: 0.0173962 
Epoch: 476 | Loss: 0.0186457 
Epoch: 477 | Loss: 0.0224893 
Epoch: 478 | Loss: 0.0282768 
Epoch: 479 | Loss: 0.0195878 
Epoch: 480 | Loss: 0.0202131 
Epoch: 481 | Loss: 0.0307570 
Epoch: 482

/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

In [22]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct/data.test_mask.sum().item()
print('Accuracy: {:.6f}'.format(acc))

Accuracy: 0.805000


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
